In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.data_preprocessing import load_data, preprocess_data
import os
import numpy as np

save_data = "save_data_2017_20_8/"

if not os.path.exists(save_data):
    os.mkdir(save_data)
    
def load_data(file_path):
    return pd.read_csv(file_path, encoding='cp1252', engine='python')
cicids2017_path = 'data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'

if(not os.path.isfile(save_data + "web_attacks.csv")):
    data_2017 = load_data(cicids2017_path)
    preprocess_data(data_2017, save_data)

In [11]:
df = pd.read_csv(save_data + 'web_attacks.csv')
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)


normal_df = df[df['Label'] == 0]
attack_df = df[df['Label'] != 0]

num_attack = len(attack_df)

num_normal = round(2.5 * num_attack)


In [12]:
normal_df = normal_df.sample(n=num_normal, random_state=42)

# Combine and shuffle the dataset
balanced_df = pd.concat([normal_df, attack_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# Split the balanced dataset into training (60%), validation (20%), and testing sets (20%)
train_df, temp_df = train_test_split(balanced_df, test_size=0.4, random_state=42, stratify=balanced_df['Label'])
val_df, test_df = train_test_split(temp_df, test_size=0.7, random_state=42, stratify=temp_df['Label'])

# Within the training set, ensure normal instances are twice the number of attack instances
train_normal_df = train_df[train_df['Label'] == 0]
train_attack_df = train_df[train_df['Label'] == 1]
num_train_attack = len(train_attack_df)
num_train_normal = 2 * num_train_attack
train_normal_df = train_normal_df.sample(n=num_train_normal, random_state=42)
train_df = pd.concat([train_normal_df, train_attack_df]).sample(frac=1, random_state=42).reset_index(drop=True)

In [13]:
# Print the count of each class in the training, validation, and testing sets
for name, df in zip(["Training", "Validation", "Testing"], [train_df, val_df, test_df]):
    unique, counts = np.unique(df['Label'], return_counts=True)
    print(f"{name} set class distribution:", dict(zip(unique, counts)))

Training set class distribution: {0: 2616, 1: 1308}
Validation set class distribution: {0: 654, 1: 261}
Testing set class distribution: {0: 1526, 1: 611}


In [14]:
# Save the train, validation, and test sets to CSV files
train_df.to_csv(save_data +  'train_set.csv', index=False)
val_df.to_csv(save_data +  'val_set.csv', index=False)
test_df.to_csv(save_data + 'test_set.csv', index=False)

In [15]:
# Define feature columns (excluding the label column)
feature_cols = [col for col in df.columns if col != 'Label']

# Extract features and labels for training, validation, and testing sets
X_train = train_df[feature_cols]
y_train = train_df['Label']
X_val = val_df[feature_cols]
y_val = val_df['Label']
X_test = test_df[feature_cols]
y_test = test_df['Label']

In [16]:
# Define classifiers
from models.decision_tree import train_decision_tree
from models.random_forest import train_random_forest
from models.lda import train_lda
from models.naive_bayes import train_naive_bayes
from models.logistic_regression import train_logistic_regression
from models.knn import train_knn
from models.svm import train_svm
from models.extra_trees import train_extra_trees
from models.bagging import train_bagging
from models.mlp import train_mlp

classifiers = {
    'Decision Tree': train_decision_tree,
    'Random Forest': train_random_forest,
    'Linear Discriminant Analysis': train_lda,
    'Naive Bayes': train_naive_bayes,
    'Logistic Regression': train_logistic_regression,
    'K-Nearest Neighbors': train_knn,
    'Support Vector Machine': train_svm,
    'Extra Trees Classifier': train_extra_trees,
    'Bagging Classifier': train_bagging,
    'Multi-layer Perceptron': train_mlp
}

In [17]:
def model_10Classifier():
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train, y_train, X_val, y_val, X_test, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            "Test Accuracy": test_acc,
            "Test Precision": test_pre,
            "Test Recall": test_rec,
            "Test F1 Score": test_f1,
            'Time': time
        })

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(save_data + '10Classsifer_results_2017.csv', index=False)
    
model_10Classifier()

Decision Tree Validation - Accuracy: 0.9759562841530055 Precision: 0.9578544061302682 Recall: 0.9578544061302682 F1 Score: 0.9578544061302682
Decision Tree Test - Accuracy: 0.9742629854936827 Precision: 0.968013468013468 Recall: 0.9410801963993454 F1 Score: 0.9543568464730291
Testing time: 0.0063 seconds
Random Forest Validation - Accuracy: 0.9825136612021858 Precision: 0.9657794676806084 Recall: 0.9731800766283525 F1 Score: 0.9694656488549618
Random Forest Test - Accuracy: 0.9789424426766495 Precision: 0.9685430463576159 Recall: 0.9574468085106383 F1 Score: 0.9629629629629629
Testing time: 0.0259 seconds
LDA Validation - Accuracy: 0.9639344262295082 Precision: 0.956 Recall: 0.9157088122605364 F1 Score: 0.9354207436399217
LDA Test - Accuracy: 0.9518015910154423 Precision: 0.9334470989761092 Recall: 0.8952536824877251 F1 Score: 0.9139515455304928
Testing time: 0.0050 seconds
Navi_bayes Validation - Accuracy: 0.8459016393442623 Precision: 0.65 Recall: 0.9961685823754789 F1 Score: 0.78668

c:\Users\a\anaconda3\envs\AI\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic_regression Validation - Accuracy: 0.9672131147540983 Precision: 0.9714285714285714 Recall: 0.9118773946360154 F1 Score: 0.9407114624505929
Logistic_regression Test - Accuracy: 0.9597566682264858 Precision: 0.9629629629629629 Recall: 0.8936170212765957 F1 Score: 0.9269949066213922
Testing time: 0.0053 seconds
KNN Validation - Accuracy: 0.9704918032786886 Precision: 0.9398496240601504 Recall: 0.9578544061302682 F1 Score: 0.9487666034155597
KNN Test - Accuracy: 0.9639681796911558 Precision: 0.9405940594059405 Recall: 0.9328968903436988 F1 Score: 0.9367296631059984
Testing time: 0.0864 seconds
SVM Validation - Accuracy: 0.9431693989071038 Precision: 0.9130434782608695 Recall: 0.8850574712643678 F1 Score: 0.8988326848249028
SVM Test - Accuracy: 0.9377632194665418 Precision: 0.9078498293515358 Recall: 0.8707037643207856 F1 Score: 0.8888888888888888
Testing time: 0.0823 seconds
Extra_trees Validation - Accuracy: 0.9814207650273225 Precision: 0.9692307692307692 Recall: 0.9655172413793

In [18]:
import pandas as pd
from sklearn.decomposition import PCA

# Assume the classifiers dictionary and train functions are already defined
# Define a function to apply PCA and train classifiers

def train_with_pca(X_train, y_train, X_val, y_val, X_test, y_test, n_components=20):
    X_combined = np.concatenate((X_train, X_val, X_test), axis=0)
    y_combined = np.concatenate((y_train, y_val, y_test), axis=0)

    pca = PCA(n_components=n_components)
    X_combined_pca = pca.fit_transform(X_combined)

    # Bước 3: Tách lại dữ liệu thành các tập train, validation và test
    X_train_pca = X_combined_pca[:len(X_train)]
    X_val_pca = X_combined_pca[len(X_train):len(X_train)+len(X_val)]
    X_test_pca = X_combined_pca[len(X_train)+len(X_val):]

    # Kiểm tra kích thước của các tập dữ liệu sau khi PCA
    print("X_train_pca shape:", X_train_pca.shape)
    print("X_val_pca shape:", X_val_pca.shape)
    print("X_test_pca shape:", X_test_pca.shape)
    
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train_pca, y_train, X_val_pca, y_val, X_test_pca, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1,
            'Time': time
        })
    
    return results

# Call the function and save results
pca_results = train_with_pca(X_train, y_train, X_val, y_val, X_test, y_test)
pca_results_df = pd.DataFrame(pca_results)
pca_results_df.to_csv(save_data + 'pca_model_results_2017.csv', index=False)


X_train_pca shape: (3924, 20)
X_val_pca shape: (915, 20)
X_test_pca shape: (2137, 20)
Decision Tree Validation - Accuracy: 0.9792349726775956 Precision: 0.9689922480620154 Recall: 0.9578544061302682 F1 Score: 0.9633911368015414
Decision Tree Test - Accuracy: 0.9700514740290126 Precision: 0.9550748752079867 Recall: 0.939443535188216 F1 Score: 0.9471947194719472
Testing time: 0.0040 seconds
Random Forest Validation - Accuracy: 0.9825136612021858 Precision: 0.9693486590038314 Recall: 0.9693486590038314 F1 Score: 0.9693486590038314
Random Forest Test - Accuracy: 0.9719232569021994 Precision: 0.956882255389718 Recall: 0.9443535188216039 F1 Score: 0.9505766062602965
Testing time: 0.0072 seconds
LDA Validation - Accuracy: 0.9398907103825137 Precision: 0.9401709401709402 Recall: 0.842911877394636 F1 Score: 0.8888888888888888
LDA Test - Accuracy: 0.9321478708469817 Precision: 0.9412878787878788 Recall: 0.8134206219312602 F1 Score: 0.8726953467954346
Testing time: 0.0053 seconds
Navi_bayes Valid

c:\Users\a\anaconda3\envs\AI\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic_regression Validation - Accuracy: 0.9453551912568307 Precision: 0.9011406844106464 Recall: 0.9080459770114943 F1 Score: 0.9045801526717557
Logistic_regression Test - Accuracy: 0.9363593823116518 Precision: 0.8964941569282137 Recall: 0.8788870703764321 F1 Score: 0.8876033057851239
Testing time: 0.0050 seconds
KNN Validation - Accuracy: 0.9704918032786886 Precision: 0.9398496240601504 Recall: 0.9578544061302682 F1 Score: 0.9487666034155597
KNN Test - Accuracy: 0.9630322882545624 Precision: 0.9389438943894389 Recall: 0.9312602291325696 F1 Score: 0.9350862777321282
Testing time: 0.0189 seconds
SVM Validation - Accuracy: 0.9431693989071038 Precision: 0.9130434782608695 Recall: 0.8850574712643678 F1 Score: 0.8988326848249028
SVM Test - Accuracy: 0.9377632194665418 Precision: 0.9078498293515358 Recall: 0.8707037643207856 F1 Score: 0.8888888888888888
Testing time: 0.0679 seconds
Extra_trees Validation - Accuracy: 0.9792349726775956 Precision: 0.9689922480620154 Recall: 0.9578544061302

In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Định nghĩa Autoencoder
# Định nghĩa Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        encoding_dim1 = 52
        encoding_dim2 = 26
        encoding_dim3 = 10

        # Lớp mã hóa
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim1, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim3),
            nn.BatchNorm1d(encoding_dim3),  # Thêm BatchNorm1d
            nn.ReLU()
        )
        # Lớp giải mã
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim3, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim1, input_dim),
            nn.Sigmoid()  # Giữ sigmoid cho đầu ra
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Hàm để áp dụng deep autoencoder và train classifiers
def train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, epochs=10, batch_size=256, alpha1=0.1):
    input_dim = X_train.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Khởi tạo mô hình và các thành phần cần thiết
    autoencoder = Autoencoder(input_dim).to(device)
    optimizer = optim.AdamW(autoencoder.parameters(), lr=0.0001)
    criterion = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop
    for epoch in range(epochs):
        autoencoder.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            decode, encode = autoencoder(inputs)
            loss = alpha1 * criterion(decode, inputs)  # Chỉ sử dụng MSE loss
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), max_norm=1.0)  # Gradient Clipping
            optimizer.step()
            train_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader)}')

    autoencoder.eval()
    with torch.no_grad():
        X_train_enc = autoencoder.encoder(X_train_tensor.to(device)).cpu().numpy()
        X_val_enc = autoencoder.encoder(X_val_tensor.to(device)).cpu().numpy()
        X_test_enc = autoencoder.encoder(torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)).cpu().numpy()
    
    # Train và đánh giá classifiers
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train_enc, y_train, X_val_enc, y_val, X_test_enc, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1,
            'Time': time
        })
    
    return results

# Gọi hàm và lưu kết quả
autoencoder_results = train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test)
autoencoder_results_df = pd.DataFrame(autoencoder_results)
autoencoder_results_df.to_csv(save_data + 'autoencoder_model_results.csv', index=False)


Epoch [1/10], Loss: 2295354245120.0
Epoch [2/10], Loss: 2445693001728.0
Epoch [3/10], Loss: 2317712887808.0
Epoch [4/10], Loss: 2332424499200.0
Epoch [5/10], Loss: 2452755494912.0
Epoch [6/10], Loss: 2307110490112.0
Epoch [7/10], Loss: 2288217627648.0
Epoch [8/10], Loss: 2357063245824.0
Epoch [9/10], Loss: 2341927985152.0
Epoch [10/10], Loss: 2426820730880.0
Decision Tree Validation - Accuracy: 0.9693989071038251 Precision: 0.939622641509434 Recall: 0.9540229885057471 F1 Score: 0.9467680608365019
Decision Tree Test - Accuracy: 0.9644361254094524 Precision: 0.948073701842546 Recall: 0.9263502454991817 F1 Score: 0.9370860927152318
Testing time: 0.0036 seconds
Random Forest Validation - Accuracy: 0.9759562841530055 Precision: 0.9578544061302682 Recall: 0.9578544061302682 F1 Score: 0.9578544061302682
Random Forest Test - Accuracy: 0.9672437997192326 Precision: 0.947107438016529 Recall: 0.9378068739770867 F1 Score: 0.9424342105263158
Testing time: 0.0141 seconds
LDA Validation - Accuracy: 0

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import random
from sklearn import preprocessing  # Import the preprocessing module from sklearn


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Cố định seed
set_seed(42)

# Định nghĩa Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        encoding_dim1 = 60
        encoding_dim2 = 50
        encoding_dim3 = 40
        encoding_dim4 = 20
        encoding_dim5 = 12

        # Lớp mã hóa
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim1, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim2, encoding_dim3),
            nn.BatchNorm1d(encoding_dim3),  # Thêm BatchNorm1d
            nn.Linear(encoding_dim3, encoding_dim4),
            nn.BatchNorm1d(encoding_dim4),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim4, encoding_dim5),
            nn.BatchNorm1d(encoding_dim5),  # Thêm BatchNorm1d
            nn.Tanh(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim5, encoding_dim4),
            nn.BatchNorm1d(encoding_dim4),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim4, encoding_dim3),
            nn.BatchNorm1d(encoding_dim3),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim3, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim2, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim1, input_dim), 
            nn.Tanh(),
        )
        self._initialize_weights()
    
    def _initialize_weights(self):
        for layer in self.encoder:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
        for layer in self.decoder:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Định nghĩa BinaryMagnetLoss
class BinaryMagnetLoss(nn.Module):
    def __init__(self, D=10, M=2, alpha=12.18, epsilon=0):
        super(BinaryMagnetLoss, self).__init__()
        self.D = D
        self.M = M
        self.alpha = alpha
        self.epsilon = epsilon

    def forward(self, outputs, indices, assignment):
        outputs = outputs.float()
        unique_clusters = torch.unique(assignment[indices])
        batch_clusters = {cluster.item(): (assignment[indices] == cluster).nonzero(as_tuple=True)[0] for cluster in unique_clusters}

        num_instances = sum(len(batch_clusters[cluster.item()]) for cluster in unique_clusters)
        c_means = torch.stack([outputs[batch_clusters[cluster.item()]].mean(dim=0) for cluster in unique_clusters])

        stdev = sum((outputs[i] - c_means[unique_clusters.tolist().index(assignment[indices[i]])]).norm(p=2).pow(2) 
                    for i in range(len(indices))) / max(num_instances - 1, 1)
        stdev = -2.0 * (stdev + self.epsilon)

        if torch.isnan(stdev).any():
            stdev = torch.tensor(1.0, device=outputs.device)  # Gán giá trị hợp lý nếu phát hiện NaN

        loss = torch.zeros(1).to(outputs.device)
        denom = [torch.zeros(1).to(outputs.device) for _ in range(len(indices))]

        for m, cluster in enumerate(unique_clusters):
            cluster_indices = batch_clusters[cluster.item()]
            for d in range(min(self.D, len(cluster_indices))):
                ind = cluster_indices[d].item()
                for other_cluster in unique_clusters:
                    if other_cluster != cluster:
                        denom[ind] += ((outputs[ind] - c_means[unique_clusters.tolist().index(other_cluster)]).norm().pow(2) / stdev).exp()

                loss_term = (((outputs[ind] - c_means[m]).norm().pow(2) / stdev - self.alpha).exp() / (denom[ind] + self.epsilon)).log().clamp(max=0.0)
                if torch.isnan(loss_term).any() or torch.isinf(loss_term).any():
                    continue
                loss -= loss_term

        loss /= num_instances
        return loss

# Custom Loss Function combining Cross-Entropy and Binary Magnet Loss
def custom_loss(y_true, y_pred, embeddings, inputs, alpha1=1, alpha2=1):
    mse_loss = F.mse_loss(inputs, y_pred) 
    magnet_loss_fn = BinaryMagnetLoss()
    magnet_loss_value = magnet_loss_fn(embeddings, torch.arange(len(y_true)), y_true)
    return alpha1 * magnet_loss_value + alpha2 * mse_loss

# Hàm để áp dụng deep autoencoder và train classifiers
def train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, classifiers, save_data, epochs, batch_size=256, alpha1=1, alpha2=1):
    input_dim = X_train.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    autoencoder = Autoencoder(input_dim).to(device)
    optimizer = optim.Adadelta(autoencoder.parameters(), lr=1)

    
    scaler = preprocessing.MaxAbsScaler()
    scaler.fit(X_train)  # Fit the scaler on the training data
    
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        autoencoder.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            decoded, encoded = autoencoder(inputs)
            loss = custom_loss(labels, decoded, encoded, inputs, alpha1=alpha1, alpha2=alpha2)
            if torch.isnan(loss).any() or torch.isinf(loss).any():
                print(f'Invalid loss detected at epoch {epoch+1}. Skipping this batch.')
                continue
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), max_norm=1.0)  # Gradient Clipping
            optimizer.step()
            train_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader)}')

    autoencoder.eval()
    with torch.no_grad():
        X_train_enc = autoencoder.encoder(X_train_tensor.to(device)).cpu().numpy()
        X_val_enc = autoencoder.encoder(X_val_tensor.to(device)).cpu().numpy()
        X_test_enc = autoencoder.encoder(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().numpy()

    results = []
    try:
        for name, train_function in classifiers.items():
            model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train_enc, y_train, X_val_enc, y_val, X_test_enc, y_test, save_data)
            #if(test_acc >= 0.979):
            results.append({
                'Model': name,
                'Validation Accuracy': val_acc,
                'Validation Precision': val_pre,
                'Validation Recall': val_rec,
                'Validation F1 Score': val_f1,
                'Test Accuracy': test_acc,
                'Test Precision': test_pre,
                'Test Recall': test_rec,
                'Test F1 Score': test_f1,
                'epochs': epochs,
                'alpha1': alpha1,
                'alpha2': alpha2,
                'Time': time
            })
    except Exception as e:
        print(f"Error: {e}")

    return results

def generate_random_choice():
    num1 = random.randint(0, 10)
    num2 = random.randint(0, 10)
    return num1, num2

def generate_random_choice1():
    values = [i/100 for i in range(1, 11)]
    num1 = random.choice(values)
    num2 = random.choice(values)
    return num1, num2

def generate_random_choice2():
    values = [i/10 for i in range(1, 11)]
    num1 = random.choice(values)
    num2 = random.choice(values)
    return num1, num2

def generate_random_choice3():
    values1 = [i/10 for i in range(1, 11)]
    values2 = [i/100 for i in range(1, 11)]
    num1 = random.choice(values1)
    num2 = random.choice(values2)
    return num1, num2

def generate_random_choice4():
    values1 = [i/100 for i in range(1, 11)]
    values2 = [i/10 for i in range(1, 11)]
    num1 = random.choice(values1)
    num2 = random.choice(values2)
    return num1, num2

maxx  = 50
for rd in range(1, maxx):
    if rd < maxx/5:
        alpha1, alpha2 = generate_random_choice()
    elif rd < 2*maxx/5:
        alpha1, alpha2 = generate_random_choice1()
    if rd < 3*maxx/5:
        alpha1, alpha2 = generate_random_choice2()
    elif rd < 4*maxx/5:
        alpha1, alpha2 = generate_random_choice3()
    else:
        alpha1, alpha2 = generate_random_choice4()
# alpha1 = 1
# alpha2 = 0.1

    autoencoder_results = train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, classifiers, save_data, epochs=6, alpha1=alpha1, alpha2=alpha2)
    autoencoder_results_df = pd.DataFrame(autoencoder_results)

    csv_file = save_data + 'autoencoder_Binary_Magnet_Loss_results_2017_20_8_1_10_12.csv'

    with open(csv_file, mode='a+') as file:
        file.write('\n')  # Thêm một dòng trống
    autoencoder_results_df.to_csv(csv_file, mode='a+', header=True, index=False)

Epoch [1/6], Loss: 0.07194050855468959
Epoch [2/6], Loss: 0.021372704184614122
Epoch [3/6], Loss: 0.013443655509036034
Epoch [4/6], Loss: 0.01265883824089542
Epoch [5/6], Loss: 0.011097619455540553
Epoch [6/6], Loss: 0.009407650795765221
Decision Tree Validation - Accuracy: 0.9748633879781421 Precision: 0.9612403100775194 Recall: 0.9501915708812261 F1 Score: 0.9556840077071291
Decision Tree Test - Accuracy: 0.9700514740290126 Precision: 0.9565943238731218 Recall: 0.9378068739770867 F1 Score: 0.947107438016529
Testing time: 0.0040 seconds
Random Forest Validation - Accuracy: 0.980327868852459 Precision: 0.9584905660377359 Recall: 0.9731800766283525 F1 Score: 0.9657794676806084
Random Forest Test - Accuracy: 0.9747309312119794 Precision: 0.9588138385502472 Recall: 0.9525368248772504 F1 Score: 0.9556650246305419
Testing time: 0.0129 seconds
LDA Validation - Accuracy: 0.9650273224043716 Precision: 0.9598393574297188 Recall: 0.9157088122605364 F1 Score: 0.9372549019607843
LDA Test - Accurac